In [4]:
import pandas as pd
import ssl 
import numpy as np
import os
import requests
import logging
from bs4 import BeautifulSoup
from pathlib import Path
import time

ssl._create_default_https_context = ssl._create_unverified_context


Table: 'player_overview'

In [5]:
def player_overview(p_id):
    # URL of the website to scrape
    url = "https://www.espncricinfo.com/cricketers/x-"+p_id

    # Send an HTTP GET request to the website
    response = requests.get(url)

    # Parse the HTML code using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract the relevant information from the HTML code
    class_grid='ds-grid lg:ds-grid-cols-3 ds-grid-cols-2 ds-gap-4 ds-mb-8'
    div_p = soup.find('div', {'class': class_grid})
    
    my_dict = {}

    class_para = 'ds-text-tight-m ds-font-regular ds-uppercase ds-text-typo-mid3'
    for div in div_p.find_all('div'):
        if ( div.find('p', {'class':class_para}) ) is None:
            continue

        key = div.find('p', {'class':class_para}).text
        value = div.find('span').text
        my_dict[key] = value

    #Adds players' roll ids: (ALL for All rounder), (BAT for Batsman), (BOW for Bowler)
    if 'All' in my_dict.get('Playing Role'):
        my_dict['Roll_id'] = 'ALL'
    elif 'Bowl' in my_dict.get('Playing Role'):
        my_dict['Roll_id'] = 'BOW'
    elif 'Bat' in my_dict.get('Playing Role'):
        my_dict['Roll_id'] = 'BAT'
    else:
        log.error('No matching playing role is found')

    player_overview = pd.DataFrame.from_dict([my_dict])
    player_overview.insert(0, "P_id", p_id, True)

    filepath = Path('Csv/player_overview.csv')
    hdr = False  if os.path.isfile(filepath) else True
    player_overview.to_csv(filepath, mode='a', index=False, header=hdr)

    player_overview

Table: 'player_pic'

#Main function below:

In [6]:
#253802: Virat Kohli (BAT)
#34102: Rohit Gurunath Sharma (BAT)
#422108: Kannaur Lokesh Rahul (BAT)
#625383: Jasprit Jasbirsingh Bumrah (BOW)
#481896: Mohammed Shami Ahmed (BOW)
#26421: Ravichandran Ashwin (BOW)
#234675: Ravindrasinh Anirudhsinh Jadeja (ALL)
#625371: Hardik Himanshu Pandya (ALL)

player_ids=['253802', '34102', '422108', '625383', '481896', '26421', '234675', '625371']

os.system("rm -f Csv/player_overview.csv")

for pid in player_ids:
    player_overview(pid)
    time.sleep(1)
